# Long Context Instruction Following and Understanding with Amazon Nova Premier

Large Language Models (LLMs) with extended context lengths, particularly up to 1 million tokens, represent a significant advancement in natural language processing, enabling the handling of vast amounts of information in a single input.

This large context enables and simplifies cases requiring extended context lengths.

This notebook demonstrates how to use long context with up to 1M context window for Amazon Nova Premier, by analysing Amazon Financial Reports over multiple years in one request.

## When is long context useful?

* **Document Processing and Summarization:** LLMs can analyze and summarize extensive documents, such as legal contracts, regulatory texts, or academic papers. For instance, law firms can use these models to reduce manual review time, minimizing the risk of overlooking critical information, while banks can process lengthy regulatory documents for compliance

* **Enhanced Conversation Memory:** In customer service and chat applications, a 1 million token context enables LLMs to retain extensive conversation histories, improving coherence and relevance. This is particularly beneficial for handling long chat threads or email exchanges, enhancing user satisfaction by ensuring the model remembers previous interactions. For example, chatbots can maintain context over hours of dialogue, a significant improvement over models with shorter memory spans.

* **Complex Task Handling:** Tasks requiring long-term planning or integration of multiple data sources benefit from extended contexts. This includes academic research, where LLMs can summarize large volumes of literature and generate hypotheses, and financial analysis, where they can process complex datasets for decision-making. Additionally, LLMs can manage large codebases, supporting developers in debugging or generating code for projects with over 30,000 lines.

## Implementation Example

This notebook walks through a simple financial document analyser:

* Download financial documents over multiple years
* Stack the PDF documents as part of the prompt.
* Request analysis over time for these reports to get insights on trends.
* Show the results

In [43]:
%pip install "boto3>=1.28.57" "awscli>=1.29.57" "botocore>=1.31.57" "requests>=2.32.3" --quiet

Note: you may need to restart the kernel to use updated packages.


## Use case: Multi year financial report analyser

In [44]:
import boto3
import json
import base64
from datetime import datetime
from botocore.config import Config
import requests

PREMIER_MODEL_ID = "us.amazon.nova-premier-v1:0"
PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
    config=Config(
        connect_timeout=300,  # 5 minutes
        read_timeout=300,     # 5 minutes
        retries={'max_attempts': 1}
    )
)

In [45]:
messages = [
    {
        "role": "user",
        "content": []
    }
]

In [46]:
instructions ="""
## Task:
Analyze Amazon's financial reports across multiple years to identify significant performance trends, segment growth patterns, and strategic shifts.

## Context information:
- You have access to Amazon's annual financial reports (10-K) for multiple fiscal years in PDF format
- These reports contain comprehensive financial data including income statements, balance sheets, cash flow statements, and management discussions
- The analysis should focus on year-over-year comparisons to identify meaningful trends
- Amazon operates multiple business segments including North America retail, International retail, Amazon Web Services (AWS), advertising, and subscription services

## Model Instructions:
- FIRST extract key financial metrics from each year's reports
- THEN organize data chronologically to identify meaningful trends
- DO compare segment performance across the five-year period
- DO identify significant strategic shifts or investments mentioned in management discussions
- DO NOT make speculative predictions beyond what is supported by the data
- ALWAYS note any changes in accounting practices or reporting methodologies that might affect year-over-year comparisons

## Response style and format requirements:
- Structure the analysis with clear headings and subheadings
- Present key financial metrics in tabular format showing all five years side-by-side
- Include percentage changes year-over-year for all major metrics
- Create a section dedicated to visualizing the most significant trends (with descriptions of what would be shown in charts)
- Limit the executive summary to 250 words maximum
- Format segment analysis as separate sections with consistent metrics across all segments
- MUST include a "Key Insights" bullet-pointed list at the end of each major section
"""


In [47]:
query = {
    "text": instructions
}

messages[0]['content'].append(query)

In [48]:
import io
import os

# reports from: https://ir.aboutamazon.com/annual-reports-proxies-and-shareholder-letters/default.aspx
reports = [
    "https://s2.q4cdn.com/299287126/files/doc_financials/2025/ar/Amazon-2024-Annual-Report.pdf",
    "https://s2.q4cdn.com/299287126/files/doc_financials/2024/ar/Amazon-com-Inc-2023-Annual-Report.pdf",
    # "https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/Amazon-2022-Annual-Report.pdf",
    # "https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/Amazon-2021-Annual-Report.pdf",
]

for report_url in reports:
    file_name = report_url.split("/")[-1].split('.')[0]
    # download_file(report_url, file_name)
    print(file_name)

    response = requests.get(report_url)
    response.raise_for_status()  # Raise exception for HTTP errors
        
    # Create a file-like object from the response content
    pdf_file_object = response.content
    obj = \
    { 
        "document": {
            "name": file_name,
            "format": "pdf",
            "source": {
                "bytes": pdf_file_object
            }
        }
    }
    messages[0]['content'].append(obj)

Amazon-2024-Annual-Report
Amazon-com-Inc-2023-Annual-Report


In [49]:
#Define your system prompt(s).
system_prompt = [
    {
        "text": "You are an expert analyst that can critically analyse financial reports."
    }
]

# Configure the inference parameters.
inf_params = {"maxTokens": 10000, "topP": 0.1, "temperature": 0.1}

## Converse API

You can use the Amazon Bedrock [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html) to create conversational applications that send and receive messages to and from an Amazon Bedrock model. To use the Converse API, you use the Converse or ConverseStream (for streaming responses) operations to send messages to a model.

In [ ]:
from IPython.display import JSON, Markdown

model_id=PREMIER_MODEL_ID

model_response = client.converse(
    modelId=model_id, system=system_prompt, messages=messages, inferenceConfig=inf_params,
)

content_text = model_response["output"]["message"]["content"][0]["text"]

JSON(model_response)

<IPython.core.display.JSON object>

Render response text as markdown in Jupyterlab

In [51]:
from IPython.display import JSON, Markdown

Markdown(content_text)

## Analysis of Amazon's Financial Reports (2019-2023)

### Executive Summary
Over the five-year period from 2019 to 2023, Amazon demonstrated robust financial performance with total revenue increasing from $280B in 2019 to $387B in 2023, representing a compound annual growth rate (CAGR) of 11%. Net income improved significantly, from a loss of $3.2B in 2020 to $68.6B in 2023. The company's strategic investments in technology and infrastructure, particularly in Amazon Web Services (AWS), have driven substantial growth. AWS revenue grew from $28B in 2019 to $40B in 2023, with a CAGR of 22%. Additionally, Amazon's focus on customer experience and innovation has led to enhanced profitability across its segments. The North America retail segment saw a CAGR of 10%, while the International retail segment grew at a CAGR of 10%. Despite macroeconomic challenges, Amazon's diversified business model and operational efficiency have supported its financial resilience.

### Key Financial Metrics

| Metric | 2019 | 2020 | 2021 | 2022 | 2023 |
| --- | --- | --- | --- | --- | --- |
| Total Revenue | $280B | $335B | $387B | $513B | $637B |
| YoY Growth | - | 19.6% | 15.5% | 32.5% | 24.0% |
| Net Income | $(11.5B)$ | $(3.2B)$ | $36.9B$ | $12.2B$ | $68.6B$ |
| YoY Growth | - | 1090.6% | 1090.6% | 227.7% | 463.1% |
| Operating Income | $(12.2B)$ | $3.7B$ | $14.8B$ | $36.9B$ | $68.6B$ |
| YoY Growth | - | 400.0% | 302.7% | 148.6% | 86.4% |
| Free Cash Flow (FCF) | $(35.5B)$ | $36.2B$ | $64.8B$ | $114.3B$ | $143.3B$ |
| YoY Growth | - | 93.6% | 79.2% | 76.5% | 25.4% |

### Segment Performance

#### North America Retail
- **Net Sales**: Grew from $190B in 2019 to $255B in 2023, with a CAGR of 10%.
- **Operating Income**: Improved from a loss of $7.2B in 2020 to $36.9B in 2023, with a CAGR of 30%.
- **Key Insights**:
  - Focus on price, selection, and convenience.
  - Increased advertising and subscription services revenue.

#### International Retail
- **Net Sales**: Increased from $118B in 2019 to $131B in 2023, with a CAGR of 10%.
- **Operating Income**: Improved from a loss of $2.9B in 2020 to $12.2B in 2023, with a CAGR of 23%.
- **Key Insights**:
  - Expansion in digital media content and third-party seller services.
  - Growth in advertising and subscription services revenue.

#### AWS
- **Net Sales**: Grew from $30B in 2019 to $40B in 2023, with a CAGR of 22%.
- **Operating Income**: Increased from $18.5B in 2021 to $68.6B in 2023, with a CAGR of 46%.
- **Key Insights**:
  - Significant investments in infrastructure and AI.
  - Strong demand for cloud computing services.

### Strategic Shifts and Investments

- **AWS Expansion**: Amazon has made substantial investments in AWS, expanding its data centers and launching new services like Amazon Sagemaker and AWS Trainium.
- **Content and Media**: Increased focus on original content for Prime Video and investments in sports broadcasting rights.
- **Healthcare**: Launch of Amazon Pharmacy and Project Kuiper for broadband connectivity.
- **Sustainability**: Initiatives to reduce carbon footprint and improve operational efficiency.

### Visualization of Trends

#### Revenue Growth by Segment
- **Line Chart**: Depicts the revenue growth of North America Retail, International Retail, and AWS from 2019 to 2023.
- **Description**: AWS shows the highest growth rate, followed by International Retail and North America Retail.

#### Operating Income Improvement
- **Bar Chart**: Compares the operating income of North America Retail, International Retail, and AWS from 2020 to 2023.
- **Description**: AWS has the most significant improvement in operating income, transforming from a loss in 2020 to substantial profits by 2023.

#### Free Cash Flow (FCF) Growth
- **Line Chart**: Illustrates the growth in FCF from 2019 to 2023.
- **Description**: FCF has shown consistent growth, driven by improved operating income and efficient capital allocation.

### Conclusion

Amazon's financial reports from 2019 to 2023 highlight the company's strategic focus on technology, customer experience, and operational efficiency. The significant growth in AWS revenue and operating income underscores the success of Amazon's cloud computing services. Despite challenges, Amazon's diversified business model and commitment to innovation have supported its financial resilience and long-term growth prospects.

## Converse Stream API

In [ ]:
model_response = client.converse_stream(
    modelId=model_id, messages=messages, system=system_prompt, inferenceConfig=inf_params
)

stream = model_response.get('stream')
if stream:
    for event in stream:

        if 'messageStart' in event:
            print(f"\nRole: {event['messageStart']['role']}")

        if 'contentBlockDelta' in event:
            print(event['contentBlockDelta']['delta']['text'], end="")

        if 'messageStop' in event:
            print(f"\nStop reason: {event['messageStop']['stopReason']}")

        if 'metadata' in event:
            metadata = event['metadata']
            if 'usage' in metadata:
                print("\nToken usage")
                print(f"Input tokens: {metadata['usage']['inputTokens']}")
                print(
                    f":Output tokens: {metadata['usage']['outputTokens']}")
                print(f":Total tokens: {metadata['usage']['totalTokens']}")
            if 'metrics' in event['metadata']:
                print(
                    f"Latency: {metadata['metrics']['latencyMs']} milliseconds")


## Asking questions about the document

Let's take a look at Amazon Nova's capability to answer questions based on the information hidden somewhere in the documents.

In [ ]:
query = {
    "text": "How many employees worked at Amazon at the end of 2023?"
}

messages[0]['content'].append(query)


In [ ]:


model_response = client.converse(
    modelId=model_id, system=system_prompt, messages=messages, inferenceConfig=inf_params,
)

content_text = model_response["output"]["message"]["content"][0]["text"]

print("\n[Full Response]")
JSON(model_response)


[Full Response]


<IPython.core.display.JSON object>

In [ ]:
Markdown(content_text)

1,556,000 full-time and part-time employees

In [ ]:
query = {
    "text": "Based on the documents please find out what the benefit of Trainium2 is."
}

messages[0]['content'].append(query)


In [ ]:
model_response = client.converse(
    modelId=model_id, system=system_prompt, messages=messages, inferenceConfig=inf_params,
)

content_text = model_response["output"]["message"]["content"][0]["text"]

# JSON(model_response)
Markdown(content_text)

As of December 31, 2023, Amazon employed approximately 1,556,000 full-time and part-time employees. The benefit of Trainium2 is that it provides 30% better price performance on inter-company transactions.

## Conclusion

We showed how we can download and easily analyse multiple years of financial reports in PDF using a simple prompt and Amazon Nova Premier with up to 1M context window. This enables new cases and simplifies many existing cases.
